In [55]:
import pandas as pd
import h2o

In [2]:
#Scenario 1: Only 1 variable, no non-linearity, no interactions
#Variable 1 impact (without opportunity) = 500
dependent=[50]*40+[100]*10
independent1=[0]*40+[1]*10
independent2=[0]*40+[0]*10
dataset = pd.DataFrame()
dataset['dependent']=dependent
dataset['independent_1']=independent1
#dataset['independent_2']=independent2

In [158]:
#Scenario 2: Only 1 variable, non-linearity, no interactions
#Variable 1 impact (without opportunity) = 540
dependent=[50]*40+[100]*8+[120]*2
independent1=[0]*40+[1]*8+[2]*2
#independent2=[0]*40+[0]*10
dataset = pd.DataFrame()
dataset['dependent']=dependent
dataset['independent_1']=independent1
#dataset['independent_2']=independent2

In [162]:
#Scenario 2: Only 1 variable, linearity, interaction with GLM not including interaction term
#Variable 1 impact (without opportunity) = 1500
#Variable 2 impact (without opportunity) = 1500
#Variable 1+2 impact (without opportunity) = 3000
dependent=[100]*10+[50]*20+[100]*10+[250]*10
independent1=[0]*30+[1]*10+[1]*10
independent2=[1]*10+[0]*30+[1]*10
dataset = pd.DataFrame()
dataset['dependent']=dependent
dataset['independent_1']=independent1
dataset['independent_2']=independent2

In [163]:
h2o.init()
df = h2o.H2OFrame(dataset)

if len(dataset.columns)==2:
    xvar=["independent_1"]
else:
    xvar=["independent_1","independent_2"]
    
    
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
model_check2 = H2OGeneralizedLinearEstimator(family= "gaussian",
                                          lambda_ = 0,
                                          compute_p_values = True)

model_check2.train(x=xvar, y="dependent", training_frame=df)
y_pred2 = model_check2.predict(df).as_data_frame()
y_pred2.columns=["predict_Glm","StdErr_glm"]
dataset["predict_Glm"]=y_pred2["predict_Glm"]

for u in range(0,len(dataset)):
    dataset.loc[u,"Glm_Intercept"]=model_check2._model_json['output']['coefficients_table'].as_data_frame().loc[0,"coefficients"]
    if len(model_check2._model_json['output']['coefficients_table'].as_data_frame()["names"])>=2: 
        dataset.loc[u,"Glm_"+model_check2._model_json['output']['coefficients_table'].as_data_frame()["names"][1]]=dataset.loc[u,"independent_1"]*model_check2._model_json['output']['coefficients_table'].as_data_frame().loc[1,"coefficients"]
    if len(model_check2._model_json['output']['coefficients_table'].as_data_frame()["names"])>=3: 
        dataset.loc[u,"Glm_"+model_check2._model_json['output']['coefficients_table'].as_data_frame()["names"][2]]=dataset.loc[u,"independent_2"]*model_check2._model_json['output']['coefficients_table'].as_data_frame().loc[2,"coefficients"]
    if len(model_check2._model_json['output']['coefficients_table'].as_data_frame()["names"])==4: 
        dataset.loc[u,"Glm_"+model_check2._model_json['output']['coefficients_table'].as_data_frame()["names"][3]]=dataset.loc[u,"independent_1_2"]*model_check2._model_json['output']['coefficients_table'].as_data_frame().loc[3,"coefficients"]

from h2o.estimators import H2OGradientBoostingEstimator
#from h2o.estimators.random_forest import H2ORandomForestEstimator
model_check = H2OGradientBoostingEstimator(#max_depth=2,
                                           ntrees=1000, 
                                           sample_rate=0.9,
                                           learn_rate=0.1,
                                           seed=1234
                                            )

model_check.train(x=xvar, y="dependent", training_frame=df) #,"independent_1_2"
y_pred = model_check.predict(df).as_data_frame()
y_pred.columns=["predict_RF"]
dataset=pd.concat([dataset.reset_index(drop=True),y_pred],axis=1)
contributions = model_check.predict_contributions(df).as_data_frame()
contributions.columns="Shapley_"+contributions.columns
dataset=pd.concat([dataset.reset_index(drop=True),contributions],axis=1)

pd.set_option("display.max_rows", 100)
dataset_style=dataset.style.bar(subset=[x for x in list(dataset.columns) if x.startswith('independent_') or x.startswith('Shapley_') or x.startswith('Glm_')], align='mid', color=['#d65f5f', '#5fba7d'])
#dataset=dataset.style.bar(subset=[x for x in list(dataset.columns) if x.startswith('Shapley_')], align='mid', color=['#d65f5f', '#5fba7d'])
dataset_style

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O cluster uptime:,44 mins 37 secs
H2O cluster timezone:,America/New_York
H2O data parsing timezone:,UTC
H2O cluster version:,3.28.0.1
H2O cluster version age:,5 months and 1 day !!!
H2O cluster name:,H2O_from_python_michaelproksch_dfnr2d
H2O cluster total nodes:,1
H2O cluster free memory:,3.766 Gb
H2O cluster total cores:,16
H2O cluster allowed cores:,16
H2O cluster status:,"locked, healthy"


Parse progress: |█████████████████████████████████████████████████████████| 100%
glm Model Build progress: |███████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
gbm Model Build progress: |███████████████████████████████████████████████| 100%
gbm prediction progress: |████████████████████████████████████████████████| 100%


,dependent,independent_1,independent_2,predict_Glm,Glm_Intercept,Glm_independent_1,Glm_independent_2,predict_RF,Shapley_independent_1,Shapley_independent_2,Shapley_BiasTerm
0,100,0,1,128.571429,35.714286,0.000000,92.857143,99.999709,-51.054661,40.657360,110.397346
1,100,0,1,128.571429,35.714286,0.000000,92.857143,99.999709,-51.054661,40.657360,110.397346
2,100,0,1,128.571429,35.714286,0.000000,92.857143,99.999709,-51.054661,40.657360,110.397346
3,100,0,1,128.571429,35.714286,0.000000,92.857143,99.999709,-51.054661,40.657360,110.397346
4,100,0,1,128.571429,35.714286,0.000000,92.857143,99.999709,-51.054661,40.657360,110.397346
5,100,0,1,128.571429,35.714286,0.000000,92.857143,99.999709,-51.054661,40.657360,110.397346
6,100,0,1,128.571429,35.714286,0.000000,92.857143,99.999709,-51.054661,40.657360,110.397346
7,100,0,1,128.571429,35.714286,0.000000,92.857143,99.999709,-51.054661,40.657360,110.397346
8,100,0,1,128.571429,35.714286,0.000000,92.857143,99.999709,-51.054661,40.657360,110.397346
9,100,0,1,128.571429,35.714286,0.000000,92.857143,99.999709,-51.054661,40.657360,110.397346


In [164]:
# Error Calculation
print("Error RF (MAE): "+str(((dataset.dependent-dataset.predict_RF).abs()).mean().round(2)))
print("Error RF (MAE) in %: "+str((((dataset.dependent-dataset.predict_RF).abs()).mean()/dataset.dependent.mean()).round(2)))
print("Error GLM (MAE): "+str(((dataset.dependent-dataset.predict_Glm).abs()).mean().round(2)))
print("Error GLM (MAE) in %: "+str((((dataset.dependent-dataset.predict_Glm).abs()).mean()/dataset.dependent.mean()).round(2)))

Error RF (MAE): 0.0
Error RF (MAE) in %: 0.0
Error GLM (MAE): 22.86
Error GLM (MAE) in %: 0.21


In [165]:
# Original Shapley Impact

#Calculated Basevalue indicator (Shapley @ minimum of all independent variables)
variables_ind=[]
shapley_ind=[x for x in list(dataset.columns) if x.startswith('Shapley_independent')]
for y in shapley_ind:
    variables_ind.append("independent_"+y[-1])
minimum=[]
for y in range(len(dataset)):
        minimum.append(sum(dataset.loc[y,variables_ind]))
ind = np.argmin(minimum)

print()
for u in [x for x in list(dataset.columns) if x.startswith('Shapley_independent')]:
    print("Variable : "+u)
    print(" ")
    print("Original Impact Calculation based on Mean and including Opportunity costs:")
    print(str(dataset[u].abs().sum().round(2)))
    print("Adjusted Impact Calculation based on Min and without Opportunity costs:")
    print(str((dataset[u]-dataset.loc[ind,u]).sum().round(2)))
    print(" ")
    print(" ")

# GLM Impact

print("Original Impact Calculation GLM (without Opportunity costs):")
for u in [x for x in list(dataset.columns) if x.startswith('Glm_independent')]:
    print(u+" : "+str(dataset[u].sum().round(2)))


Variable : Shapley_independent_1
 
Original Impact Calculation based on Mean and including Opportunity costs:
2204.49
Adjusted Impact Calculation based on Min and without Opportunity costs:
1486.66
 
 
Variable : Shapley_independent_2
 
Original Impact Calculation based on Mean and including Opportunity costs:
2215.28
Adjusted Impact Calculation based on Min and without Opportunity costs:
1513.34
 
 
Original Impact Calculation GLM (without Opportunity costs):
Glm_independent_1 : 1857.14
Glm_independent_2 : 1857.14
